In [ ]:
# teste
#import torch
#import gc

# After you are done with a model or tensors
#del model  # Delete the model instance
#del results_train  # Delete any large tensor variables

#gc.collect()        # Run Python's garbage collector
#torch.cuda.empty_cache() # Clear the CUDA cache

In [ ]:
#!git clone https://github.com/williancarddd/ocr-table-recognition.git

In [ ]:
#!pwd

In [ ]:
#!cp -a /content/ocr-table-recognition/. /content

In [ ]:
#from ultralytics import YOLO
#from pathlib import Path

# YOLo

In [ ]:
!pip3 install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.4 MB/s eta 0:00:00


In [ ]:
import os
import json
import shutil
from pathlib import Path

In [ ]:
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import torch

In [ ]:
from google.colab import drive
from google.colab import runtime

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_path_dataset = Path("/content/drive/MyDrive/dataset_icdar_completo/")
#drive_path_dataset = Path("/content/drive/MyDrive/dataset_folds_sem_aumento/")
#drive_path_dataset = Path("/content/drive/MyDrive/dataset_folds/")
#drive_path_dataset = Path("/content/drive/MyDrive/sampled_resized_fintabnet/")

path_dataset = Path("/content/dataset_completo/")
#path_dataset = Path("/content/dataset_folds_sem_aumento/")
#path_dataset = Path("/content/dataset_folds/")
#path_dataset = Path("/content/sampled_resized_fintabnet/")

In [ ]:
drive_path_to_save = Path('/content/drive/MyDrive/modelos_salvos/')

In [ ]:
#shutil.rmtree('runs_yolo_folds_all')
#shutil.rmtree(path_dataset)

In [ ]:
# executar somente com reinicialização do ambiente de execução (não é sessão)
# transfere os dados do Drive para a VM do Colab para facilitar operações de I/O

#shutil.copytree(drive_path_dataset, path_dataset, copy_function=copy_with_progress)
shutil.copytree(drive_path_dataset, path_dataset)

PosixPath('/content/dataset_completo')

In [ ]:
print('Finalizou...')

Finalizou...


# Segmentação

In [ ]:
model_name = 'yolov8m-seg.pt'
project_name = "runs_yolo_folds_all"

epochs = 10
imgsz = 640
batch = 32
optimizer = 'adam'
lr=1e-3 # Padrão do Adam
plots = True
cache = True
workers = 16
save = True
augmentations=[]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
warmup_epochs = 2
close_mosaic = 0

print(f"Treinando modelo: {model_name}\n")
print(f'Dispositivo: {device}')

for fold_idx in range(1, 6):

  fold_path = path_dataset/f'fold_{fold_idx}/'
  data_yaml_path = fold_path/"dataset.yaml"

  # Nome do experimento (separa por modelo e fold)
  #run_name = f"{Path(model_name).stem}_fold_{fold_idx}"
  run_name = f"{model_name}_fold_{fold_idx}"
  path_to_save_run = f'/content/{project_name}/{run_name}'
  drive_path_to_save_run = drive_path_to_save/model_name/run_name

  print("YAML:", data_yaml_path)

  # Carrega o modelo correspondente
  model = YOLO(model_name)

  results_train = model.train(
      data=str(data_yaml_path),
      epochs=epochs,
      imgsz=imgsz,
      batch=batch,
      optimizer=optimizer,
      lr0=lr,
      warmup_epochs = warmup_epochs,
      project=project_name,
      name=run_name,
      plots = plots,
      augmentations=augmentations,
      device=device,
      cache=cache,
      workers=workers,
      save=save,
      close_mosaic=close_mosaic,
      exist_ok=True
  )

  with open(f"{path_to_save_run}/results2.json", "w") as file:
    results_json = results_train.to_json()
    json.dump(results_json, file)

  shutil.copytree(path_to_save_run, drive_path_to_save_run)
  print(f"Treinamento do fold {fold_idx} com {model_name} concluído.")


# destroi o ambiente quando terminar de executar
runtime.unassign()

Treinando modelo: yolov8m-seg.pt

Dispositivo: cuda
YAML: /content/dataset_folds_sem_aumento/fold_1/dataset.yaml
engine/trainer: agnostic_nms=False, amp=True, augment=False, augmentations=[], auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=0, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_folds_sem_aumento/fold_1/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m-seg.pt_fold_1, nbs=64, nms=False, opset=None, optimize

In [ ]:
#runtime.unassign()

# Detecção

In [ ]:

model_name = 'yolov8x.pt'
project_name = "runs_yolo_folds_all"



epochs = 10
imgsz = 640
batch = 32
optimizer = 'adam'
lr=1e-3 # Padrão do Adam
plots = True
cache = True
workers = 16
save = True
augmentations=[]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
warmup_epochs = 2
close_mosaic = 0
fold_idx = 1



print(f"Treinando modelo: {model_name}\n")
print(f'Dispositivo: {device}')

data_yaml_path = path_dataset/"dataset.yaml"

# Nome do experimento (separa por modelo e fold)
#run_name = f"{Path(model_name).stem}_fold_{fold_idx}"
run_name = f"{model_name}_fold_{fold_idx}"
path_to_save_run = f'/content/{project_name}/{run_name}'
drive_path_to_save_run = drive_path_to_save/model_name/run_name

print("YAML:", data_yaml_path)

# Carrega o modelo correspondente
model = YOLO(model_name)

results_train = model.train(
    data=str(data_yaml_path),
    epochs=epochs,
    imgsz=imgsz,
    batch=batch,
    optimizer=optimizer,
    lr0=lr,
    warmup_epochs = warmup_epochs,
    project=project_name,
    name=run_name,
    plots = plots,
    augmentations=augmentations,
    device=device,
    cache=cache,
    workers=workers,
    save=save,
    close_mosaic=close_mosaic,
    exist_ok=True
)

with open(f"{path_to_save_run}/results2.json", "w") as file:
  results_json = results_train.to_json()
  json.dump(results_json, file)

shutil.copytree(path_to_save_run, drive_path_to_save_run)
print(f"Treinamento do {model_name} concluído.")


# destroi o ambiente quando terminar de executar
#runtime.unassign()

Treinando modelo: yolov8m-seg.pt

Dispositivo: cuda
YAML: /content/dataset_completo/dataset.yaml
engine/trainer: agnostic_nms=False, amp=True, augment=False, augmentations=[], auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=0, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_completo/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m-seg.pt_fold_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=adam, overlap_

In [ ]:
1+1

2

# MaskRCNN (TESTE)

Teste realizado com o Mask R-CNN, mas que não foi desenvolvido.

In [ ]:
!git clone https://github.com/pytorch/vision.git


Cloning into 'vision'...
remote: Enumerating objects: 698465, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 698465 (delta 177), reused 60 (delta 60), pack-reused 698238 (from 3)
Receiving objects: 100% (698465/698465), 1.22 GiB | 16.45 MiB/s, done.
Resolving deltas: 100% (654237/654237), done.


In [ ]:
!cd vision; cp references/detection/utils.py ../;  # Copy utility functions
!cd vision; cp references/detection/transforms.py ../;  # Copy image transformation functions
!cd vision; cp references/detection/coco_eval.py ../;  # Copy COCO evaluation script
!cd vision; cp references/detection/engine.py ../;  # Copy engine code for training/evaluation
!cd vision; cp references/detection/coco_utils.py ../;  # Copy COCO-specific utility functions

In [ ]:

import os
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from pycocotools.coco import COCO

from engine import train_one_epoch, evaluate
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor



In [ ]:
#drive_path_dataset = Path("/content/drive/MyDrive/dataset_rcnn/")
#path_dataset = Path("/content/dataset_rcnn/")

In [ ]:
#shutil.copytree(drive_path_dataset, path_dataset)

PosixPath('/content/dataset_rcnn')

In [ ]:
class CocoDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, json_path):
        self.coco = COCO(json_path)
        # Filter out image IDs that do not have any annotations
        self.ids = []
        for img_id in list(self.coco.imgs.keys()):
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            if len(ann_ids) > 0:
                self.ids.append(img_id)

        self.img_dir = img_dir

    def __getitem__(self, index):
        img_id = self.ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        img_info = self.coco.loadImgs(img_id)[0]

        # monta caminho corretamente
        img_path = os.path.join(self.img_dir, img_info["file_name"])

        image = torchvision.io.read_image(img_path).float() / 255.0

        boxes, labels, masks, area, iscrowd = [], [], [], [], []
        for ann in anns:
            xmin, ymin, w, h = ann["bbox"]
            boxes.append([xmin, ymin, xmin + w, ymin + h])
            labels.append(ann["category_id"])
            masks.append(self.coco.annToMask(ann))
            area.append(ann["area"])
            iscrowd.append(ann.get("iscrowd", 0))

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "masks": torch.tensor(masks, dtype=torch.uint8),
            "area": torch.tensor(area, dtype=torch.float32),
            "iscrowd": torch.tensor(iscrowd, dtype=torch.int64),
            "image_id": torch.tensor([img_id])
        }

        return image, target

    def __len__(self):
        return len(self.ids)


class MaskRCNNTrainer:
    def __init__(self, root_folder, epochs=10, batch_size=2):

        self.root = root_folder
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.epochs = epochs

        # caminhos do COCO
        train_json = os.path.join(root_folder, "train.json")
        val_json = os.path.join(root_folder, "val.json")

        # pastas com imagens
        train_img_dir = os.path.join(root_folder, "train")
        val_img_dir = os.path.join(root_folder, "val")

        self.train_dataset = CocoDataset(train_img_dir, train_json)
        self.val_dataset = CocoDataset(val_img_dir, val_json)

        self.train_loader = DataLoader(
            self.train_dataset,
            batch_size=batch_size,
            shuffle=True,
            collate_fn=lambda x: tuple(zip(*x))
        )

        self.val_loader = DataLoader(
            self.val_dataset,
            batch_size=1,
            shuffle=False,
            collate_fn=lambda x: tuple(zip(*x))
        )

        # número de classes (COCO + background)
        self.num_classes = len(self.train_dataset.coco.getCatIds()) + 1

        self.model = self.create_model()


    def create_model(self):
      model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

      # box head
      in_features = model.roi_heads.box_predictor.cls_score.in_features
      model.roi_heads.box_predictor = FastRCNNPredictor(in_features, self.num_classes)

      # mask head
      in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
      model.roi_heads.mask_predictor = MaskRCNNPredictor(
          in_features_mask,
          256,
          self.num_classes
      )

      return model.to(self.device)

    def run(self):
        optimizer = torch.optim.SGD(
            [p for p in self.model.parameters() if p.requires_grad],
            lr=0.005,
            momentum=0.9,
            weight_decay=0.0005,
        )

        for epoch in range(self.epochs):
            print(f"\n=== Epoch {epoch + 1}/{self.epochs} ===")
            train_one_epoch(self.model, optimizer, self.train_loader, self.device, epoch, print_freq=25)
            evaluate(self.model, self.val_loader, device=self.device)

            torch.save(self.model.state_dict(), f"maskrcnn_epoch_{epoch+1}.pth")

        print("Treinamento finalizado!")

In [ ]:
!pwd

/content


In [ ]:

trainer = MaskRCNNTrainer(root_folder="dataset_rcnn/fold_1", epochs=1)
trainer.run()

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!

=== Epoch 1/1 ===


/tmp/ipython-input-1335682812.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  "masks": torch.tensor(masks, dtype=torch.uint8),
/content/engine.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [  0/218]  eta: 2:13:46  lr: 0.000028  loss: 8.5011 (8.5011)  loss_classifier: 0.9732 (0.9732)  loss_box_reg: 0.0000 (0.0000)  loss_mask: 0.0000 (0.0000)  loss_objectness: 6.9316 (6.9316)  loss_rpn_box_reg: 0.5962 (0.5962)  time: 36.8205  data: 34.1109  max mem: 4089
Epoch: [0]  [ 25/218]  eta: 1:22:22  lr: 0.000603  loss: 1.1092 (2.5675)  loss_classifier: 0.0318 (0.2841)  loss_box_reg: 0.0000 (0.0000)  loss_mask: 0.0000 (0.0000)  loss_objectness: 0.6003 (1.8797)  loss_rpn_box_reg: 0.3330 (0.4037)  time: 26.2634  data: 25.7292  max mem: 7433


KeyboardInterrupt: 